In [1]:
%load_ext autoreload
%autoreload 1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.decomposition import KernelPCA

In [2]:
# df_rain = pd.read_csv('datasets/benchmark/weatherAUS_clean_train.csv')
df_rain = pd.read_csv('datasets/weatherAUS_clean.csv')
rain_feats = df_rain.columns.values
rain_x = df_rain[rain_feats[1:-1]]
rain_y = df_rain[rain_feats[-1]]
rain_x_tr, rain_x_te, rain_y_tr, rain_y_te = train_test_split(rain_x, rain_y, test_size=0.2)

In [3]:
rain_x.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindGustDir_E,WindGustDir_W,WindDir9am_N,WindDir9am_S,WindDir9am_E,WindDir9am_W,WindDir3pm_N,WindDir3pm_S,WindDir3pm_E,WindDir3pm_W
0,13.4,22.9,0.6,24.666201,7.20069,44.0,20.0,24.0,71.0,22.0,...,0.0,6.0,0.0,0.0,0.0,6.0,2.0,0.0,0.0,4.0
1,7.4,25.1,0.0,24.666201,7.20069,44.0,4.0,22.0,44.0,25.0,...,0.0,4.0,4.0,0.0,0.0,2.0,0.0,2.0,0.0,4.0
2,12.9,25.7,0.0,24.666201,7.20069,46.0,19.0,26.0,38.0,30.0,...,0.0,4.0,0.0,0.0,0.0,6.0,0.0,2.0,0.0,4.0
3,9.2,28.0,0.0,24.666201,7.20069,24.0,11.0,9.0,45.0,16.0,...,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,6.0,0.0
4,17.5,32.3,1.0,24.666201,7.20069,41.0,7.0,20.0,82.0,33.0,...,0.0,6.0,2.0,0.0,4.0,0.0,3.0,0.0,0.0,3.0


In [4]:
rain_y.max()

1.0

In [5]:
import subset_pca
%aimport subset_pca

In [6]:
spca = subset_pca.SubsetPCA(type='dim', value=10)

In [13]:
spca.fit(rain_x_tr)

([{'Cloud3pm', 'Cloud9am', 'Sunshine'},
  {'RainToday', 'Rainfall'},
  {'WindDir3pm_N', 'WindDir3pm_S', 'WindGustDir_N', 'WindGustDir_S'},
  {'WindDir9am_N', 'WindDir9am_S'},
  {'WindDir3pm_E', 'WindDir3pm_W', 'WindGustDir_E', 'WindGustDir_W'},
  {'Humidity3pm', 'Humidity9am', 'MaxTemp', 'MinTemp', 'Temp3pm', 'Temp9am'},
  {'Pressure3pm', 'Pressure9am'},
  {'Evaporation'},
  {'WindGustSpeed', 'WindSpeed3pm', 'WindSpeed9am'},
  {'WindDir9am_E', 'WindDir9am_W'}],
 [PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1),
  PCA(n_components=1)])

In [14]:
small_x_tr = spca.transform(rain_x_tr)
small_x_te = spca.transform(rain_x_te)

In [15]:
from sklearn.linear_model import LogisticRegression as lr
from sklearn.metrics import confusion_matrix as cm

In [16]:
clf = lr(penalty='l2').fit(small_x_tr, rain_y_tr)
print(clf.score(small_x_tr, rain_y_tr), clf.score(small_x_te, rain_y_te))
tr_cm = cm(clf.predict(small_x_tr), rain_y_tr)
te_cm = cm(clf.predict(small_x_te), rain_y_te)
print(tr_cm[0] / sum(tr_cm[0]), tr_cm[1] / sum(tr_cm[1]))
print(te_cm[0] / sum(te_cm[0]), te_cm[1] / sum(te_cm[1]))
# print(cm(clf.predict(small_x_tr), rain_y_tr), '\n', cm(clf.predict(small_x_te), rain_y_te))

print('Coefficients:', clf.coef_)

0.8370145332129074 0.8329091058379178
[0.86009432 0.13990568] [0.30179986 0.69820014]
[0.85433637 0.14566363] [0.29529817 0.70470183]
Coefficients: [[-0.15840892  0.01812983  0.02211982 -0.0565405   0.02065933 -0.04202249
   0.05325748 -0.00254673  0.02825835 -0.02030053]]


In [17]:
clf_big = lr(penalty='l2').fit(rain_x_tr, rain_y_tr)
print(clf_big.score(rain_x_tr, rain_y_tr), clf_big.score(rain_x_te, rain_y_te))
tr_cm = cm(clf_big.predict(rain_x_tr), rain_y_tr)
te_cm = cm(clf_big.predict(rain_x_te), rain_y_te)
print(tr_cm[0] / sum(tr_cm[0]), tr_cm[1] / sum(tr_cm[1]))
print(te_cm[0] / sum(te_cm[0]), te_cm[1] / sum(te_cm[1]))

0.8472678380819443 0.8423105345266442
[0.86965298 0.13034702] [0.27872644 0.72127356]
[0.86328673 0.13671327] [0.27529156 0.72470844]


C:\ProgramData\Anaconda3\envs\env_3_6\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
